## Handling Anthropic Tool Use and Tool Result Messages

This is to test and demonstrate the intricacies of handling tool use and tool result messages in the Anthropic API.
The scenario will cover:

a) Scenario 1: tool_use message from assistant without a corresponding tool_result message from user  
b) Scenario 2: tool_result message from user that does not match the tool_use message from assistant  
c) Scenario 3: tool_result message from user that matches the tool_use message from assistant  

### a) Load Test File

In [ ]:
import os
import json
from gai.lib.tests import get_local_datadir
file_path = os.path.join(get_local_datadir(), "monologue_1.log")
with open(file_path,"r") as f:
    lines = f.read()
    jsoned = json.loads(lines)
for key, value in jsoned.items():
    if isinstance(value, list):
        print(f"{key}: {len(value)} items")
    else:
        print(f"{key}: {value}")

next_message_order: 145
messages: 145 items


### b) Scenario 1: Assistant tool_use message without User tool_result message will raise error

This is to prove that Anthropic will return error if the tool_use message is not followed by a tool_result message from the user.

In [1]:
from gai.llm.openai import AsyncOpenAI
from gai.lib.config import config_helper

chat_messages = [
    {"role":"user","content":"Something is wrong with the tests."},
    {"role": "assistant", "content": [
            {
                "id": "toolu_01LMimBJe4BVtqpxCnanoykQ",
                "input": {
                    "test_file_path": "/workspace/tests/gai-lib/minimal_test.py /workspace/tests/gai-lib/test_color.py /workspace/tests/gai-lib/test_config_config_base.py /workspace/tests/gai-lib/test_config_config_helper.py /workspace/tests/gai-lib/test_config_download_config.py /workspace/tests/gai-lib/test_config_gai_client_config.py /workspace/tests/gai-lib/test_config_gai_config.py /workspace/tests/gai-lib/test_config_gai_generator_config.py /workspace/tests/gai-lib/test_config_gai_tool_config.py /workspace/tests/gai-lib/test_constants.py /workspace/tests/gai-lib/test_errors.py /workspace/tests/gai-lib/test_logging.py /workspace/tests/gai-lib/test_prompts.py /workspace/tests/gai-lib/test_strings.py /workspace/tests/gai-lib/test_utils.py"
                },
                "name": "run_pytest",
                "type": "tool_use"
            }
        ]
    },
    {"role": "user", "content": "I am not responding with a tool_result."},
]

llm_config = config_helper.get_client_config("sonnet-4")
llm_client = AsyncOpenAI(client_config=llm_config)
tools = [
    {
        "type": "function",
        "function": {            
            "name": "run_pytest",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "input_schema": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }
]
try:
    
    response = await llm_client.chat.completions.create(
        model="sonnet-4",
        messages=chat_messages,
        tools=tools,
        stream=True,
    )
    print("Done")
except Exception as e:
    assert "`tool_use` ids were found without `tool_result` blocks immediately after: toolu_01LMimBJe4BVtqpxCnanoykQ." in str(e), f"Unexpected error: {e}"

ERROR    async_anthropic_create: Error while calling anthropic API: Error code: 400 - {'type': 'error', 'error':   
         {'type': 'invalid_request_error', 'message': 'messages.2: `tool_use` ids were found without `tool_result` 
         blocks immediately after: toolu_01LMimBJe4BVtqpxCnanoykQ. Each `tool_use` block must have a corresponding 
         `tool_result` block in the next message.'}}

### c) Scenario 2: User tool_result message does not match assistant tool_use message will raise error

This is to prove that Anthropic will return error if the tool_result message from the user does not match the tool_use message from the assistant.

In [2]:
from gai.llm.openai import AsyncOpenAI
from gai.lib.config import config_helper

chat_messages = [
    {"role": "user", "content": "Something is wrong with the tests."},
    {"role": "assistant", "content": [
        {
            "citations": None,
            "text": "Let me run the test pattern more specifically:",
            "type": "text"
        },
    ]
    },
    {"role": "user", "content": [
        {
            "type": "tool_result",
            "tool_use_id": "toolu_01LMimBJe4BVtqpxCnanoykQ",
            "content": "{\n  \"error\": \"Test file does not exist.\"\n}"
        }
    ]},
]

llm_config = config_helper.get_client_config("sonnet-4")
llm_client = AsyncOpenAI(client_config=llm_config)
tools = [
    {
        "type": "function",
        "function": {
            "name": "run_pytest",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "input_schema": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }
]
try:
    response = await llm_client.chat.completions.create(
        model="claude-sonnet-4-0",
        messages=chat_messages,
        tools=tools,
        stream=True,
    )
    print("Done")
except Exception as e:
    assert "unexpected `tool_use_id` found in `tool_result` blocks: toolu_01LMimBJe4BVtqpxCnanoykQ. Each `tool_result` block must have a corresponding `tool_use` block in the previous message." in str(e), f"Unexpected error: {e}"

ERROR    async_anthropic_create: Error while calling anthropic API: Error code: 400 - {'type': 'error', 'error':   
         {'type': 'invalid_request_error', 'message': 'messages.2.content.0: unexpected `tool_use_id` found in     
         `tool_result` blocks: toolu_01LMimBJe4BVtqpxCnanoykQ. Each `tool_result` block must have a corresponding  
         `tool_use` block in the previous message.'}}

### d) Scenario 3: Working scenario with user tool_result matching assistant tool_use

This is to demonstrate a working scenario where the user tool_result matches the assistant tool_use message.

In [4]:
from gai.llm.openai import AsyncOpenAI
from gai.lib.config import config_helper

chat_messages = [
    {"role": "user", "content": "Something is wrong with the tests."},
    {"role": "assistant", "content": [
        {
            "id": "toolu_01LMimBJe4BVtqpxCnanoykQ",
            "input": {
                "test_file_path": "/workspace/tests/gai-lib/minimal_test.py /workspace/tests/gai-lib/test_color.py /workspace/tests/gai-lib/test_config_config_base.py /workspace/tests/gai-lib/test_config_config_helper.py /workspace/tests/gai-lib/test_config_download_config.py /workspace/tests/gai-lib/test_config_gai_client_config.py /workspace/tests/gai-lib/test_config_gai_config.py /workspace/tests/gai-lib/test_config_gai_generator_config.py /workspace/tests/gai-lib/test_config_gai_tool_config.py /workspace/tests/gai-lib/test_constants.py /workspace/tests/gai-lib/test_errors.py /workspace/tests/gai-lib/test_logging.py /workspace/tests/gai-lib/test_prompts.py /workspace/tests/gai-lib/test_strings.py /workspace/tests/gai-lib/test_utils.py"
            },
            "name": "run_pytest",
            "type": "tool_use"
        }
    ]
    },
    {"role": "user", "content": [
        {
            "type": "tool_result",
            "tool_use_id": "toolu_01LMimBJe4BVtqpxCnanoykQ",
            "content": "{\n  \"error\": \"Test file does not exist.\"\n}"
        }
    ]},
]

llm_config = config_helper.get_client_config("sonnet-4")
llm_client = AsyncOpenAI(client_config=llm_config)
tools = [
    {
        "type": "function",
        "function": {
            "name": "run_pytest",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "input_schema": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }
]
try:

    response = await llm_client.chat.completions.create(
        model="claude-sonnet-4-0",
        messages=chat_messages,
        tools=tools,
        stream=True,
    )
    print("Done")
except Exception as e:
    assert "`tool_use` ids were found without `tool_result` blocks immediately after: toolu_01LMimBJe4BVtqpxCnanoykQ." in str(
        e), f"Unexpected error: {e}"

Done
